In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
import os
import random
from scrape_utils import get_tm_team_links, get_tm_team_squad

In [ ]:
TEAMS_DIR = os.path.join('data', 'transfermarkt', 'teams')
PLAYERS_DIR = os.path.join('data', 'transfermarkt', 'players')
os.makedirs(TEAMS_DIR, exist_ok=True)
os.makedirs(PLAYERS_DIR, exist_ok=True)

# Get the links for each football team in the leagues for the 2023-24 season

In [ ]:
league_urls = ['https://www.transfermarkt.com/bundesliga/startseite/wettbewerb/L1/plus/?saison_id=2023',
               'https://www.transfermarkt.com/2-bundesliga/startseite/wettbewerb/L2/plus/?saison_id=2023',
               'https://www.transfermarkt.com/premier-league/startseite/wettbewerb/GB1/plus/?saison_id=2023',
               'https://www.transfermarkt.com/championship/startseite/wettbewerb/GB2/plus/?saison_id=2023',
               'https://www.transfermarkt.com/serie-a/startseite/wettbewerb/IT1/plus/?saison_id=2023',
               'https://www.transfermarkt.com/serie-b/startseite/wettbewerb/IT2/plus/?saison_id=2023',
               'https://www.transfermarkt.com/laliga/startseite/wettbewerb/ES1/plus/?saison_id=2023',
               'https://www.transfermarkt.com/laliga2/startseite/wettbewerb/ES2/plus/?saison_id=2023',
               'https://www.transfermarkt.com/ligue-1/startseite/wettbewerb/FR1/plus/?saison_id=2023',
               'https://www.transfermarkt.com/ligue-2/startseite/wettbewerb/FR2/plus/?saison_id=2023',
               'https://www.transfermarkt.com/eredivisie/startseite/wettbewerb/NL1/plus/?saison_id=2023',
               'https://www.transfermarkt.com/jupiler-pro-league/startseite/wettbewerb/BE1/plus/?saison_id=2023',
               'https://www.transfermarkt.com/liga-nos/startseite/wettbewerb/PO1/plus/?saison_id=2023',
               'https://www.transfermarkt.com/major-league-soccer/startseite/wettbewerb/MLS1/plus/?saison_id=2023',
               'https://www.transfermarkt.com/campeonato-brasileiro-serie-a/startseite/wettbewerb/BRA1/plus/?saison_id=2023',
               'https://www.transfermarkt.com/liga-mx-clausura/startseite/wettbewerb/MEX1/plus/?saison_id=2023',
               'https://www.transfermarkt.com/superliga/startseite/wettbewerb/AR1N/plus/?saison_id=2023',
              ]
team_links = []
team_league = []
years = []
for url in league_urls:
    links = get_tm_team_links(url)
    team_links.extend(links)
    team_league.extend([url.split('/')[3]] * len(links))
    years.extend([int(url[-4:]) + 1] * len(links))
    time.sleep(random.randint(2, 5))

team_links = [s.replace('spielplan', 'kader') for s in team_links] 
team_links = [f'{l}/plus/1' for l in team_links]
df_links = pd.DataFrame({'league': team_league, 'year': years, 'url': team_links})
df_links.to_parquet(os.path.join(TEAMS_DIR, 'teams.parquet'))

# Get squad data

In [ ]:
df_links = pd.read_parquet(os.path.join(TEAMS_DIR, 'teams.parquet'))
print(len(df_links))

In [ ]:
for index, row in df_links.iterrows():
    df_squad = get_tm_team_squad(row.url)
    df_squad['year'] = row.year
    df_squad['league'] = row.league
    FILE_NAME = f'{row.url.split("/")[3]}.parquet'
    df_squad.to_parquet(os.path.join(PLAYERS_DIR, FILE_NAME))
    time.sleep(random.randint(2, 5))